Notebook Setup

In [7]:
#import dependencies
import requests
from pprint import pprint
from requests.auth import HTTPBasicAuth
import pandas as pd

#authentication flow
client_id = "4b454f446d1f479082eec9de1c27a4a8"
client_secret = "e639450422594948b17c46d1a82f4882"

url = "https://accounts.spotify.com/api/token?grant_type=client_credentials"
headers = {'content-type': 'application/x-www-form-urlencoded'}

try:
   response = requests.post(url, headers=headers, auth=(client_id, client_secret))
   json = response.json()
   print(json)
   access_token = json["access_token"]
   print(access_token)
except:
   print("Something went wrong with the request")

{'access_token': 'BQBcqxO7U6c4NsuvH-mIiaciAw0nXCIX4U2k2jikBgggmgO3o_X8RLI3DkxSgD09KbIkn4vze4u4OoWWbYg', 'token_type': 'Bearer', 'expires_in': 3600, 'scope': ''}
BQBcqxO7U6c4NsuvH-mIiaciAw0nXCIX4U2k2jikBgggmgO3o_X8RLI3DkxSgD09KbIkn4vze4u4OoWWbYg


Collect Data

In [8]:
# https://api.spotify.com/v1/artists/{id}

#This is for US Top 50 playlist
playlist_id = "37i9dQZEVXbLRQDuF5jeBp"
#spotify:user:spotifycharts:playlist:37i9dQZEVXbLRQDuF5jeBp

url = f"https://api.spotify.com/v1/playlists/{playlist_id}"

headers = {'authorization': f"Bearer {access_token}"}
response = requests.get(url, headers=headers)

#pprint(response.json())
#print(json.dumps(newrelease, sort_keys=True, indent=4))
jj = response.json()
#pprint(jj)

In [87]:
#test check: 
pprint(jj['tracks']['items'][0]['track']['album']['uri'][14:])

'2fYhqwDWXjbpjaIJPEfKFw'


In [10]:
songs= jj['tracks']['items']

In [95]:
#Create lists for relevant data & then organize into a dataframe
ids = []
artists = []
tracks = []
releasedate = []
audiofeatures = []
popularity = []
for x in songs:
    idsong = x["track"]["id"]
    ids.append(idsong)
    artist = x['track']['album']['artists'][0]['name']
    artists.append(artist)
    track = x['track']['name']
    tracks.append(track)
    rd = x["track"]["album"]["release_date"]
    releasedate.append(rd)
    poppin = x['track']['popularity']
    popularity.append(poppin)
    url = f"https://api.spotify.com/v1/audio-features/{idsong}"
    headers = {'authorization': f"Bearer {access_token}"}
    response = requests.get(url, headers=headers)
    features= response.json()
    audiofeatures.append(features)
#pprint(features)

{'error': {'message': 'The access token expired', 'status': 401}}


In [88]:
#looking for genre of each artist
albumid = []
genre = []
for x in songs:
    ida = x['track']['album']['uri'][14:]
    albumid.append(ida)
    url = f"https://api.spotify.com/v1/albums/{ida}"
    headers = {'authorization': f"Bearer {access_token}"}
    response = requests.get(url, headers=headers)
    gen= response.json()

#pprint(gen)

{'error': {'message': 'The access token expired', 'status': 401}}


In [54]:
#separate the dictionary into lists
afkeys = [list(x.keys()) for x in audiofeatures]
afvalues = [list(x.values()) for x in audiofeatures]

#create a dataframe for audio features
af = pd.DataFrame(afvalues, columns=afkeys[0])
#create a dataframe for track info
tdata = pd.DataFrame(list(zip(tracks, artists, releasedate, popularity)),columns=["tracks", "artists","released", "popularity"])

In [55]:
tdata.head()

,tracks,artists,released,popularity
0,7 rings,Ariana Grande,2019-02-08,91
1,"break up with your girlfriend, i'm bored",Ariana Grande,2019-02-08,96
2,MIDDLE CHILD,J. Cole,2019-01-23,94
3,Wow.,Post Malone,2018-12-24,98
4,Sunflower - Spider-Man: Into the Spider-Verse,Various Artists,2018-12-14,98


In [89]:
#combine the two dataframes 
data = pd.concat([tdata, af], axis=1, sort=False)

#clean data by removing unneccesary columns
data.drop(["uri", "track_href", "analysis_url"],axis = 1, inplace = True)
data

,tracks,artists,released,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,duration_ms,time_signature
0,7 rings,Ariana Grande,2019-02-08,91,0.771,0.320,1,-10.752,0,0.3160,0.60100,0.000000,0.0885,0.312,139.844,audio_features,6ocbgoVGwYJhOv1GgI9NsF,178627,4
1,"break up with your girlfriend, i'm bored",Ariana Grande,2019-02-08,96,0.726,0.545,5,-5.489,0,0.1020,0.03780,0.000000,0.1080,0.348,169.991,audio_features,4kV4N9D1iKVxx1KLvtTpjS,190440,4
2,MIDDLE CHILD,J. Cole,2019-01-23,94,0.837,0.364,8,-11.713,1,0.2760,0.14900,0.000000,0.2710,0.463,123.984,audio_features,2JvzF1RMd7lE3KmFlsyZD8,213594,4
3,Wow.,Post Malone,2018-12-24,98,0.833,0.539,11,-7.399,0,0.1780,0.16300,0.000002,0.1010,0.385,99.947,audio_features,6MWtB6iiXyIwun0YzU6DFP,149520,4
4,Sunflower - Spider-Man: Into the Spider-Verse,Various Artists,2018-12-14,98,0.760,0.479,2,-5.574,1,0.0466,0.55600,0.000000,0.0703,0.913,89.911,audio_features,3KkXRkHbMCARz0aVfEt68P,158040,4
5,Robbery,Juice WRLD,2019-02-13,78,0.708,0.691,2,-5.179,1,0.0442,0.34200,0.000000,0.2220,0.535,79.990,audio_features,44ShclXTOOnYtt5CnCDeGV,240527,4
6,Murder On My Mind,YNW Melly,2018-08-03,86,0.759,0.730,0,-7.985,0,0.0516,0.14500,0.000003,0.1100,0.740,115.007,audio_features,7eBqSVxrzQZtK2mmgRG6lC,268434,4
7,Going Bad (feat. Drake),Meek Mill,2018-11-30,94,0.889,0.496,4,-6.365,0,0.0905,0.25900,0.000000,0.2520,0.544,86.003,audio_features,2IRZnDFmlqMuOrYOLnZZyc,180522,4
8,a lot,21 Savage,2018-12-21,92,0.837,0.636,1,-7.643,1,0.0860,0.03950,0.001250,0.3420,0.274,145.972,audio_features,2t8yVaLvJ0RenpXUIAC52d,288624,4
9,Thotiana,Blueface,2018-12-18,90,0.906,0.382,10,-12.890,0,0.2690,0.18000,0.000000,0.1130,0.391,104.025,audio_features,3oh6SCCeLuXhFpEyepla6G,129264,4


In [ ]:
#wanted to find a list of all playlists in the Top 50 Charts by Country category, but such category does not exist in the API.
#alternate resolution: manually copy and paste the playlist ids from spotify
#alternate resolution: instead of comparing data from all countries, we can analze the GLOBAL TOP 50 playlist vs GLOBAL VIRAL 50 playlist

url = f"https://api.spotify.com/v1/browse/categories/charts/playlists"
headers = {'authorization': f"Bearer {access_token}"}
response = requests.get(url, headers=headers)
cate= response.json()
pprint(cate)

In [91]:
#data.to_csv("top50us.csv", encoding='utf-8', index=False)